In [ ]:
import torch

if torch.cuda.is_available():
    print(f"GPU is available: {torch.cuda.get_device_name(0)}")
else:
    print("GPU is not available.")

GPU is available: NVIDIA A100-SXM4-40GB


In [ ]:
from huggingface_hub import login
import getpass

# Prompt the user to enter the Hugging Face token securely
token = getpass.getpass("Enter your Hugging Face token: ")

# Login to Hugging Face using the token
login(token=token)

Enter your Hugging Face token: ··········


In [ ]:
# read in all doc dir under the folder path and return a list of doc content
data_folder_dir = './data/scraped/scraped_text_data_append_sharding_3072_600_max_3072'

doc_list = []
# extract all doc content
import os
for root, dirs, files in os.walk(data_folder_dir):
    for file in files:
        if file.endswith('.txt'):
            doc_path = os.path.join(root, file)
            doc_list.append(doc_path)

print(doc_list[:5])
print(len(doc_list))

# # randomely select 10 doc to test
# doc_list = doc_list[:20]

['./data/scraped/scraped_text_data_append_sharding_3072_600_max_3072/0-2-0.txt', './data/scraped/scraped_text_data_append_sharding_3072_600_max_3072/0-14-0.txt', './data/scraped/scraped_text_data_append_sharding_3072_600_max_3072/0-16-0.txt', './data/scraped/scraped_text_data_append_sharding_3072_600_max_3072/0-18-0.txt', './data/scraped/scraped_text_data_append_sharding_3072_600_max_3072/0-0-0.txt']
764


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
# load the model
# model_id = "meta-llama/Llama-2-7b-chat-hf"
model_id = "meta-llama/Llama-3.1-8B-Instruct"
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16, device_map="auto")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

In [ ]:
# load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [ ]:
from transformers import pipeline
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, torch_dtype=torch.float16)

Device set to use cuda:0


In [ ]:
INSTRUCTION = """
I have provided a web-crawled document above that may be relevant to one or more topics such as
general information, history, economy, music, culture, sports, or upcoming events
related to Pittsburgh or Carnegie Mellon University. Based on this document,
generate 5~10 factual question and answer pairs that cover different types of inquiries,
such as time, events, people, locations, or numerical data.
If there are fewer pairs available, only provide the number you can find.
The questions should make sense independently of the document by including key words from the document.
For example, the question "Q: What is the fee of the event?" is not acceptable,
because words like "the event" is too vague without context of the document.
It should be sufficiently detailed as "Q: What is the fee of the event at PPG Arena on October 13?"
or "Q: What is the fee of the event that celebrates the 50th anniversary of CMU?".
Please provide concise answers without repeating the question or using complete sentences.
For example, given the question "Q: When was Carnegie Mellon University founded?",
you should only answer "A: 1900".
\n\nExamples: \n
Q: Who is Pittsburgh named after? A: William Pitt \n
Q: What famous machine learning venue had its first conference in Pittsburgh in 1980? A: ICML \n
Q: What musical artist is performing at PPG Arena on October 13? A: Billie Eilish\n\n
Before you start, please read the document above and provide the number of question and answer pairs you can find."""

In [ ]:
import re
import pandas as pd
def extract_qa_pairs(generated_text, doc_id):
    # Use regular expressions to extract all the questions and answers
    questions = re.findall(r'(?:\d*\.\s*)?Q:\s*(.*?)\s*A:', generated_text, re.DOTALL)
    answers = re.findall(r'A:\s*(.*?)(?=\s*(?:\d*\.\s*Q:|Q:|$))', generated_text, re.DOTALL)

    doc_ids = [doc_id] * len(questions)
    # Create a pandas DataFrame from the extracted questions and answers
    df = pd.DataFrame({
        'Doc_id': doc_ids,
        'Question': questions,
        'Answer': answers
    })

    # Display the resulting DataFrame
    return df

In [ ]:
# torch.cuda.empty_cache()

In [12]:
# import pandas as pd
from tqdm import tqdm
import datetime
# Read the content of a file
def read_preprocess_doc(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return file.read()
# Load the existing QA pairs from the CSV file to see what has already been processed
processed_qa_df_1 = pd.read_csv('./data/annotated/raw/generated_qa_pairs_1.csv')
# processed_qa_df_2 = pd.read_csv('../data/annotated/new_generated_qa_pairs_2.csv')
# processed_qa_df_3 = pd.read_csv('../data/annotated/new_generated_qa_pairs_3.csv')
# processed_qa_df_4 = pd.read_csv('../data/annotated/new_generated_qa_pairs_4.csv')
# processed_qa_df = pd.concat([processed_qa_df_1, processed_qa_df_2, processed_qa_df_3, processed_qa_df_4], axis=0)
processed_qa_df = pd.concat([processed_qa_df_1], axis=0)

# Get the list of already processed doc IDs (assuming doc_id is stored in a column called 'doc_id')
processed_doc_ids = processed_qa_df['Doc_id'].unique()

# Initialize an empty DataFrame for new QA pairs
all_qa_df = pd.DataFrame()

# Iterate over the documents in doc_list
for doc in tqdm(doc_list):
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    # Extract the doc_id from the document path
    doc_id = doc.split('/')[-1]

    # Check if this doc_id has already been processed
    if doc_id in processed_doc_ids:
        print(f"Skipping already processed document: {doc_id}")
        continue  # Skip the file if it has already been processed

    # Process the document
    print(f"Processing document: {doc_id}")
    content = read_preprocess_doc(doc)
    formatted_input = content + "\n\n" + INSTRUCTION + "\n\n" + "Your answer:"
    tokenized_prompt = tokenizer(formatted_input, return_tensors="pt").to(model.device)
    print(tokenized_prompt.input_ids.size(1))

    # Generate the response
    messages = [
        {"role": "user", "content": formatted_input},
    ]
    with torch.no_grad():
        result = pipe(messages, max_new_tokens=512)

    # Extract the question and answer pairs
    qa_df = extract_qa_pairs(result[0]['generated_text'][1]['content'], doc_id)
    qa_df.to_csv(f'./data/annotated/raw/new_generated_qa_pairs_{timestamp}.csv')

    # Append the new QA pairs to the all_qa_df
    all_qa_df = pd.concat([all_qa_df, qa_df], axis=0)

    # Reindex the DataFrame
    all_qa_df.reset_index(drop=True, inplace=True)

  0%|          | 0/764 [00:00<?, ?it/s]

Skipping already processed document: 0-2-0.txt
Skipping already processed document: 0-14-0.txt
Skipping already processed document: 0-16-0.txt
Skipping already processed document: 0-18-0.txt
Skipping already processed document: 0-0-0.txt
Skipping already processed document: 0-19-0.txt
Skipping already processed document: 0-21-0.txt
Skipping already processed document: 0-1-0.txt
Skipping already processed document: 0-11-0.txt
Skipping already processed document: 0-20-0.txt
Skipping already processed document: 0-15-0.txt
Skipping already processed document: 0-12-0.txt
Skipping already processed document: 0-17-0.txt
Skipping already processed document: 0-13-0.txt
Skipping already processed document: 0-10-0.txt
Skipping already processed document: 0-24-0.txt
Skipping already processed document: 0-39-0.txt
Skipping already processed document: 0-23-0.txt
Skipping already processed document: 0-25-0.txt
Skipping already processed document: 0-34-0.txt
Skipping already processed document: 0-38-0

  7%|▋         | 55/764 [00:22<04:44,  2.49it/s]

Skipping already processed document: 1-15-0.txt
Skipping already processed document: 1-4-0.txt
Skipping already processed document: 1-9-0.txt
Skipping already processed document: 1-19-0.txt
Processing document: 10-0-0.txt
1565


  8%|▊         | 60/764 [00:40<09:11,  1.28it/s]

...

100%|█████████▉| 762/764 [2:36:07<00:22, 11.12s/it]

Processing document: 9627_UF_Regulations-3-0.txt
1387


100%|█████████▉| 763/764 [2:36:16<00:10, 10.64s/it]

Processing document: 9626_Payroll_Tax_Regulations-8-0.txt
1462


100%|██████████| 764/764 [2:36:25<00:00, 12.29s/it]


In [13]:
all_qa_df.head()

,Doc_id,Question,Answer
0,10-1-0.txt,When was the HOV OK program made available?,Not available
1,10-1-0.txt,How many people were killed in a head-on colli...,Six
2,10-1-0.txt,"What roads form a roughly triangular-shaped ""b...","I-76, I-79, and I-70"
3,10-1-0.txt,What is the name of the system that aims to gu...,Pittsburgh Wayfinder System
4,10-1-0.txt,How many bridges dot the landscape of Alleghen...,"2,000"


In [14]:
# Save the new QA pairs back to the CSV file (optional)
all_qa_df.to_csv('./data/annotated/new_generated_qa_pairs_4.csv', index=False)

In [15]:
# read in the generated qa pairs and randomly select 50% of them for validation and rest for testing
import pandas as pd
import numpy as np
import random

qa_pairs = pd.read_csv('./data/annotated/QA_pairs.csv')
print(len(qa_pairs))

# convert the answer to str type
qa_pairs['Answer'] = qa_pairs['Answer'].apply(lambda x: str(x))

# for all duplicated question, compress them into one row, and make the answer a list, and all other columns take the first value
qa_pairs_deduplicated = qa_pairs.groupby('Question').agg({'Answer': list, 'Doc_id': 'first'}).reset_index()
print(len(qa_pairs_deduplicated))

8523
7876


In [16]:
# then only keep unique answers for each question, make the answer a concatenated string separated by semicolon
qa_pairs_deduplicated['Answer'] = qa_pairs_deduplicated['Answer'].apply(lambda x: '[SEP]'.join(list(set(x))))

In [17]:
# check one example where there are multiple answers
duplicate_qa_pairs = qa_pairs_deduplicated[qa_pairs_deduplicated['Answer'].apply(lambda x: '[SEP]' in x)]
print(len(duplicate_qa_pairs))
# randomly pick 20 questions to check
sample_duplicate_qa_pairs = duplicate_qa_pairs.sample(10)
print(sample_duplicate_qa_pairs)

128
                                               Question  \
972          What is the address of the Benedum Center?   
2974  What is the name of the film festival that cel...   
2263  What is the name of the arena where the Pittsb...   
4062  What is the name of the university's athletic ...   
6972            Who founded Carnegie Mellon University?   
7178  Who is the President of Carnegie Mellon Univer...   
6698               Where is the Benedum Center located?   
2867  What is the name of the exhibit showcasing lif...   
6097          When does the 2024 Piano Sale take place?   
2882  What is the name of the exhibition at the Carn...   

                                                 Answer        Doc_id  
972   2425 Liberty Ave[SEP]719 Liberty Avenue, Pitts...    66-2-0.txt  
2974                       Not found[SEP]Not found\n18.    0-25-0.txt  
2263          Consol Energy Center[SEP]Duquesne Gardens  115-14-0.txt  
4062  University Athletic Association (UAA)[SEP]Univ...   

In [18]:
# rename the column: Answer to Reference_Answers
qa_pairs_deduplicated.rename(columns={'Answer': 'Reference_Answers'}, inplace=True)
# randomly select 50% of the qa pairs for validation
qa_pairs_1 = qa_pairs_deduplicated.sample(frac=0.5, random_state=42)
qa_pairs_1.to_csv('./data/annotated/QA_pairs_1.csv', index=False)

print(len(qa_pairs_1))

# get the rest of the qa pairs for testing
qa_pairs_2 = qa_pairs_deduplicated[~qa_pairs_deduplicated['Question'].isin(qa_pairs_1['Question'])]
qa_pairs_2.to_csv('./data/annotated/QA_pairs_2.csv', index=False)

print(len(qa_pairs_2))

3938
3938


In [19]:
# compute the number of unqiue doc_id in the qa pairs
doc_ids = qa_pairs_1['Doc_id'].unique()
print(len(doc_ids))

760


In [20]:
# read in the "output/llama3_recursive_chroma_top3.csv" file
llama3_recursive_chroma_top3 = pd.read_csv('./output/tests/llama3_recursive_chroma_top3_default.csv')

In [21]:
# reset the index
llama3_recursive_chroma_top3.reset_index(drop=True, inplace=True)
llama3_recursive_chroma_top3.head()

,Question,Doc_id,Generated_Answer,Reference_Answers
0,Who is performing at Benedum Center on January 2?,89-1-0.txt,I don't know,Kansas
1,How many years does a suit have to be filed wi...,9625_Parking_Tax_Regulations-9-0.txt,Three (3) years,3 years
2,When was the Playhouse constructed?,139-0-0.txt,1897,1897
3,What is the number of major non-public compani...,0-14-0.txt,22,Several
4,What is the name of the city that Pittsburgh i...,0-6-0.txt,Allegheny City,Appalachia


In [22]:
qa_pairs_1.reset_index(drop=True, inplace=True)
qa_pairs_1.head()

,Question,Reference_Answers,Doc_id
0,Who is performing at Benedum Center on January 2?,Kansas,89-1-0.txt
1,How many years does a suit have to be filed wi...,3 years,9625_Parking_Tax_Regulations-9-0.txt
2,When was the Playhouse constructed?,1897,139-0-0.txt
3,What is the number of major non-public compani...,Several,0-14-0.txt
4,What is the name of the city that Pittsburgh i...,Appalachia,0-6-0.txt


In [23]:
# drop the Reference_Answers column in llama3_recursive_chroma_top3
llama3_recursive_chroma_top3.drop(columns=['Reference_Answers'], inplace=True)
llama3_recursive_chroma_top3['Reference_Answers'] = qa_pairs_1['Reference_Answers']
llama3_recursive_chroma_top3.head()

,Question,Doc_id,Generated_Answer,Reference_Answers
0,Who is performing at Benedum Center on January 2?,89-1-0.txt,I don't know,Kansas
1,How many years does a suit have to be filed wi...,9625_Parking_Tax_Regulations-9-0.txt,Three (3) years,3 years
2,When was the Playhouse constructed?,139-0-0.txt,1897,1897
3,What is the number of major non-public compani...,0-14-0.txt,22,Several
4,What is the name of the city that Pittsburgh i...,0-6-0.txt,Allegheny City,Appalachia


In [ ]:
# write it back to the same file
llama3_recursive_chroma_top3.to_csv('./output/tests/llama3_recursive_chroma_top3.csv', index=False)